In [16]:
from langdetect import detect, DetectorFactory
import langid
from transformers import pipeline
import fasttext


DetectorFactory.seed = 0

language_identifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

fasttext_model = fasttext.load_model('/content/lid.176.bin')


In [17]:
def detect_language_langdetect(text):
    try:
        language = detect(text)
        return language
    except:
        return "unknown"

def detect_language_langid(text):
    lang, _ = langid.classify(text)
    return lang

def detect_language_huggingface(text):
    truncated_text = text[:512]

    result = language_identifier(truncated_text)
    return result[0]['label']



def detect_language_fasttext(text):
    predictions = fasttext_model.predict(text)
    return predictions[0][0].replace('__label__', '')

def jasser_language_identification(text):
    lang_detect_result = detect_language_langdetect(text)
    lang_id_result = detect_language_langid(text)
    hf_result = detect_language_huggingface(text)
    ft_result = detect_language_fasttext(text)

    languages = {}

    for lang in [lang_detect_result, lang_id_result, hf_result, ft_result]:
        if lang in languages:
            languages[lang] += 1
        else:
            languages[lang] = 1

    most_common_language = max(languages, key=languages.get)
    return most_common_language


# **Testing:**

In [18]:
texts = {
    "Anglais": "The quick brown fox jumps over the lazy dog.",
    "Français": "Le renard brun rapide saute par-dessus le chien paresseux.",
    "Espagnol": "El zorro marrón rápido salta sobre el perro perezoso.",
    "Allemand": "Der schnelle braune Fuchs springt über den faulen Hund.",
    "Arabe": "الثعلب البني السريع يقفز فوق الكلب الكسول.",
    "Chinois": "快速的棕色狐狸跳过懒狗。",
    "Japonais": "速い茶色のキツネが怠惰な犬を飛び越えます。",
    "Russe": "Быстрая коричневая лиса прыгает через ленивую собаку.",
    "Italien": "La volpe marrone veloce salta sopra il cane pigro.",
    "Portugais": "A rápida raposa marrom pula sobre o cão preguiçoso."
}

for language, text in texts.items():
    detected_language = jasser_language_identification(text)
    print(f"Langue attendue : {language} - Langue détectée : {detected_language}")

Langue attendue : Anglais - Langue détectée : en
Langue attendue : Français - Langue détectée : fr
Langue attendue : Espagnol - Langue détectée : es
Langue attendue : Allemand - Langue détectée : de
Langue attendue : Arabe - Langue détectée : ar
Langue attendue : Chinois - Langue détectée : zh
Langue attendue : Japonais - Langue détectée : ja
Langue attendue : Russe - Langue détectée : ru
Langue attendue : Italien - Langue détectée : it
Langue attendue : Portugais - Langue détectée : pt


# **Evaluate the model:**

In [19]:
language_map = {
    "en": "Anglais",
    "fr": "Français",
    "es": "Espagnol",
    "de": "Allemand",
    "ar": "Arabe",
    "zh": "Chinois",
    "ja": "Japonais",
    "ru": "Russe",
    "it": "Italien",
    "pt": "Portugais"
}

# Fonction pour évaluer la précision de la détection de langue
def evaluate_language_detection(texts):
    correct_count = 0
    total_count = len(texts)

    for expected_language, text in texts.items():
        detected_language_code = jasser_language_identification(text)
        detected_language = language_map.get(detected_language_code, "Unknown")

        print(f"Langue attendue : {expected_language} - Langue détectée : {detected_language}")

        if detected_language == expected_language:
            correct_count += 1

    accuracy = correct_count / total_count
    print(f"Précision : {accuracy:.2%}")

# Évaluer la précision de la détection de langue
evaluate_language_detection(texts)

Langue attendue : Anglais - Langue détectée : Anglais
Langue attendue : Français - Langue détectée : Français
Langue attendue : Espagnol - Langue détectée : Espagnol
Langue attendue : Allemand - Langue détectée : Allemand
Langue attendue : Arabe - Langue détectée : Arabe
Langue attendue : Chinois - Langue détectée : Chinois
Langue attendue : Japonais - Langue détectée : Japonais
Langue attendue : Russe - Langue détectée : Russe
Langue attendue : Italien - Langue détectée : Italien
Langue attendue : Portugais - Langue détectée : Portugais
Précision : 100.00%


# **Evaluation of the model with long texts:**

In [20]:
paragraphs = {
    "Anglais": "The quick brown fox jumps over the lazy dog. It was a sunny day in the park. Birds were chirping and people were enjoying their picnics. The fox, satisfied with its jump, rested under a shady tree.",
    "Français": "Le renard brun rapide saute par-dessus le chien paresseux. C'était une journée ensoleillée au parc.Les oiseaux chantaient et les gens profitaient de leurs pique-niques. Le renard, satisfait de son saut, se reposa sous un arbre ombragé.",
    "Espagnol": "El zorro marrón rápido salta sobre el perro perezoso. Era un día soleado en el parque. Los pájaros cantaban y la gente disfrutaba de sus picnics. El zorro, satisfecho con su salto, descansó bajo un árbol sombrío.",
    "Allemand": "Der schnelle braune Fuchs springt über den faulen Hund. Es war ein sonniger Tag im Park. Die Vögel zwitscherten und die Leute genossen ihre Picknicks. Der Fuchs, zufrieden mit seinem Sprung, ruhte unter einem schattigen Baum.",
    "Arabe": "الثعلب البني السريع يقفز فوق الكلب الكسول. كان يومًا مشمسًا في الحديقة. كانت الطيور تغرد وكان الناس يستمتعون بنزهاتهم. الثعلب، راضيًا عن قفزته، استراح تحت شجرة ظليلة",

    "Chinois": "快速的棕色狐狸跳过懒狗。 这是公园里的一个阳光明媚的日子。 鸟儿在唱歌，人们在享受野餐。 狐狸满意地跳了跳，在一棵荫凉的树下休息。",
    "Japonais": " 速い茶色のキツネが怠惰な犬を飛び越えます。 公園は晴れた日でした。 鳥がさえずり、人々がピクニックを楽しんでいました。 キツネはその飛び越えに満足し、日陰の木の下で休憩しました。",
    "Russe": " Быстрая коричневая лиса прыгает через ленивую собаку. Это был солнечный день в парке. Птицы щебечут,  а люди наслаждаются пикниками. Лиса, удовлетворенная своим прыжком, отдохнула под тенью дерева.",
    "Italien": "La volpe marrone veloce salta sopra il cane pigro. Era una giornata soleggiata al parco. Gli uccelli cinguettavano e le persone stavano godendo dei loro picnic. La volpe, soddisfatta del suo salto, si riposò sotto un albero ombroso.",
    "Portugais": " A rápida raposa marrom pula sobre o cão preguiçoso. Foi um dia ensolarado no parque. Os pássaros estavam cantando  e as pessoas estavam aproveitando seus piqueniques. A raposa, satisfeita com seu salto, descansou sob uma árvore sombreada."
}

evaluate_language_detection(paragraphs)

Langue attendue : Anglais - Langue détectée : Anglais
Langue attendue : Français - Langue détectée : Français
Langue attendue : Espagnol - Langue détectée : Espagnol
Langue attendue : Allemand - Langue détectée : Allemand
Langue attendue : Arabe - Langue détectée : Arabe
Langue attendue : Chinois - Langue détectée : Chinois
Langue attendue : Japonais - Langue détectée : Japonais
Langue attendue : Russe - Langue détectée : Russe
Langue attendue : Italien - Langue détectée : Italien
Langue attendue : Portugais - Langue détectée : Portugais
Précision : 100.00%


# **Evaluation of the model with complex sentencesl:**

In [21]:
sentences = {
    "Anglais": "Artificial intelligence (AI) is transforming industries worldwide. From healthcare to finance, AI is driving innovation and improving efficiency. However, ethical concerns around AI algorithms continue to grow, highlighting the need for responsible AI development.",
    "Français": "L'intelligence artificielle (IA) transforme les industries du monde entier. De la santé à la finance, l'IA stimule l'innovation et améliore l'efficacité. Cependant, les préoccupations éthiques concernant les algorithmes d'IA ne cessent de croître, soulignant la nécessité d'un développement responsable de l'IA.",
    "Espagnol": "La inteligencia artificial (IA) está transformando industrias en todo el mundo. Desde la atención médica hasta las finanzas, la IA impulsa la innovación y mejora la eficiencia. Sin embargo, las preocupaciones éticas en torno a los algoritmos de IA continúan creciendo, destacando la necesidad de un desarrollo ético de la IA.",
    "Allemand": "  Künstliche Intelligenz (KI) transformiert weltweit Branchen. Von der Gesundheitsversorgung bis zur Finanzwirtschaft treibt KI Innovationen voran und verbessert die Effizienz.   Dennoch nehmen die ethischen Bedenken hinsichtlich KI-Algorithmen zu,   was die Notwendigkeit einer verantwortungsvollen Entwicklung von KI betont. ",
    "Chinois": "人工智能（AI）正在全球范围内改变各行各业。 从医疗保健到金融，AI推动创新并提高效率。 然而，围绕AI算法的伦理关注不断增长，突显了开发责任感AI的必要性。",
    "Japonais": "人工知能（AI）は世界中の産業を変革しています。 医療から金融まで、AIは革新を推進し、効率を向上させています。 しかし、AIアルゴリズムに関する倫理的な懸念は高まり続けており、 責任あるAIの開発の必要性を示しています。",
    "Russe": "Искусственный интеллект (ИИ) трансформирует отрасли по всему миру.  От здравоохранения до финансов, ИИ стимулирует инновации и повышает эффективность. Однако этические вопросы, связанные с алгоритмами ИИ, продолжают расти,  подчеркивая необходимость ответственного развития ИИ. ",
    "Italien": " L'intelligenza artificiale (AI) sta trasformando industrie in tutto il mondo.  Dalla sanità alle finanze, l'AI sta guidando l'innovazione e migliorando l'efficienza.  Tuttavia, le preoccupazioni etiche riguardanti gli algoritmi di AI continuano a crescere,  evidenziando la necessità di uno sviluppo responsabile dell'AI.",
    "Portugais": "  A inteligência artificial (AI) está transformando indústrias em todo o mundo.   Da saúde às finanças, a AI está impulsionando a inovação e melhorando a eficiência.   No entanto, as preocupações éticas em torno dos algoritmos de AI continuam a crescer,   destacando a necessidade de um desenvolvimento responsável de AI."
}

evaluate_language_detection(sentences)

Langue attendue : Anglais - Langue détectée : Anglais
Langue attendue : Français - Langue détectée : Français
Langue attendue : Espagnol - Langue détectée : Espagnol
Langue attendue : Allemand - Langue détectée : Allemand
Langue attendue : Chinois - Langue détectée : Chinois
Langue attendue : Japonais - Langue détectée : Japonais
Langue attendue : Russe - Langue détectée : Russe
Langue attendue : Italien - Langue détectée : Italien
Langue attendue : Portugais - Langue détectée : Portugais
Précision : 100.00%


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

class LanguageIdentificationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LanguageIdentificationModel, self).__init__()

        self.fc = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [27]:
input_size = 16
hidden_size = 128
output_size = 3

model = LanguageIdentificationModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

input_data = torch.randn(1, input_size)

optimizer.zero_grad()
output = model(input_data)
target = torch.tensor([0])
loss = criterion(output, target)
loss.backward()
optimizer.step()


torch.save(model.state_dict(), 'LanguageIdentificationModel_weights.bin')


model = LanguageIdentificationModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('LanguageIdentificationModel_weights.bin'))
model.eval()

LanguageIdentificationModel(
  (fc): Linear(in_features=16, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (softmax): LogSoftmax(dim=1)
)